In [1]:
import pandas as pd
from fullcontact import FullContactClient
import numpy as np
from imblearn.over_sampling import SMOTENC
from helpers import Data
from urllib.request import urlopen, Request
import mysql.connector
from getpass import getpass
from mysql.connector import connect
from bs4 import BeautifulSoup
import requests
import re
from helpers import individualFeatures, Data, Scraping


w = pd.read_csv(r'C:\Users\aacjp\Sales-Scientist\datasets\check6.csv').drop(['Unnamed: 0'], axis='columns')

In [2]:
w.head()

,name,landingPage,customer,domain,model,source,adspend,hardcosts,revenue,avg_polarity,...,global,percentPaid,sales,adRevenue,cac,roas,profit,profitMargin,lpc,averageOrderValue
0,clickfunnels.com,https://funnelscripts.com/funnelscripts-webclass,1,marketing,software,word of mouth,70000,5000000,9000000,0.219405,...,1270,0.353865,30303,8.958190e+06,0.038501,127.974142,3930000,0.436667,21736.00000,3006.752577
1,kinobody.com,https://go.kinobody.com/quiz/,1,fitness,info,word of mouth,210000,20000,630000,0.067753,...,88504,0.121291,9130,5.145120e+05,138.980807,2.450057,400000,0.634921,105833.00000,9.700074
2,FrankKern.com,https://frankkernbook.com/optin24883998?utm_so...,1,entrepreneurship,info,NaN,54437,18666,583310,0.000000,...,120191,0.174400,194,7.174713e+04,280.000000,1.317985,510207,87.470000,2198.00000,996.810407
3,ImpactTheory.Com,NaN,1,self-development,media,NaN,126095,8000,250000,0.288006,...,211062,0.140200,25773,3.057500e+04,5.000000,0.242476,115905,46.360000,10787.69697,297.007329
4,digitalmillionairesecrets.com,https://highticketselling.com/,1,entrepreneurship,info,NaN,139711,53333,1666667,0.345833,...,5225501,0.656100,1672,1.380500e+06,84.000000,9.881112,1473623,88.420000,113.00000,1996.005988


In [3]:
w2 = w.drop(['name', 'landingPage', 'source'], axis='columns')
w2.head()

,customer,domain,model,adspend,hardcosts,revenue,avg_polarity,avg_subjectivity,std_polarity,std_subjectivity,...,global,percentPaid,sales,adRevenue,cac,roas,profit,profitMargin,lpc,averageOrderValue
0,1,marketing,software,70000,5000000,9000000,0.219405,0.379881,0.247699,0.309339,...,1270,0.353865,30303,8.958190e+06,0.038501,127.974142,3930000,0.436667,21736.00000,3006.752577
1,1,fitness,info,210000,20000,630000,0.067753,0.318990,0.330783,0.316209,...,88504,0.121291,9130,5.145120e+05,138.980807,2.450057,400000,0.634921,105833.00000,9.700074
2,1,entrepreneurship,info,54437,18666,583310,0.000000,0.000000,0.000000,0.000000,...,120191,0.174400,194,7.174713e+04,280.000000,1.317985,510207,87.470000,2198.00000,996.810407
3,1,self-development,media,126095,8000,250000,0.288006,0.353015,0.167825,0.266983,...,211062,0.140200,25773,3.057500e+04,5.000000,0.242476,115905,46.360000,10787.69697,297.007329
4,1,entrepreneurship,info,139711,53333,1666667,0.345833,0.454167,0.000000,0.000000,...,5225501,0.656100,1672,1.380500e+06,84.000000,9.881112,1473623,88.420000,113.00000,1996.005988


In [4]:
domains = pd.get_dummies(w['domain'])
models = pd.get_dummies(w['model'])
dominantPlatforms = pd.get_dummies(w['dominantPlatform'])

In [8]:
list(dominantPlatforms.columns)

['Youtube',
 'facebook',
 'instagram',
 'linkedin',
 'other',
 'pinterest',
 'quora',
 'reddit',
 'youtube']

In [12]:
w3 = w2.drop(['domain', 'model', 'dominantPlatform'], axis='columns')
w3.head()

,customer,adspend,hardcosts,revenue,avg_polarity,avg_subjectivity,std_polarity,std_subjectivity,words,triggers,...,global,percentPaid,sales,adRevenue,cac,roas,profit,profitMargin,lpc,averageOrderValue
0,1,70000,5000000,9000000,0.219405,0.379881,0.247699,0.309339,247.00000,8.00000,...,1270,0.353865,30303,8.958190e+06,0.038501,127.974142,3930000,0.436667,21736.00000,3006.752577
1,1,210000,20000,630000,0.067753,0.318990,0.330783,0.316209,1163.00000,13.00000,...,88504,0.121291,9130,5.145120e+05,138.980807,2.450057,400000,0.634921,105833.00000,9.700074
2,1,54437,18666,583310,0.000000,0.000000,0.000000,0.000000,2140.00000,55.00000,...,120191,0.174400,194,7.174713e+04,280.000000,1.317985,510207,87.470000,2198.00000,996.810407
3,1,126095,8000,250000,0.288006,0.353015,0.167825,0.266983,716.40625,8.84375,...,211062,0.140200,25773,3.057500e+04,5.000000,0.242476,115905,46.360000,10787.69697,297.007329
4,1,139711,53333,1666667,0.345833,0.454167,0.000000,0.000000,108.00000,1.00000,...,5225501,0.656100,1672,1.380500e+06,84.000000,9.881112,1473623,88.420000,113.00000,1996.005988


In [14]:
w4 = pd.concat([w3, domains, models, dominantPlatforms], axis='columns')
w4.shape

(59, 57)

In [15]:
w4.to_csv(r'C:\Users\aacjp\Sales-Scientist\datasets\leadsML.csv')